In [1]:
from __future__ import print_function
from __future__ import division

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [3]:
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
from PIL import Image
import matplotlib.pyplot as plt

In [5]:
import time
import os
import copy
from tqdm.notebook import tqdm

In [6]:
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.7.0
Torchvision Version:  0.8.1


In [7]:
photo_dir = '../input/yelp-academic-photos/yelp_academic/photos/'
csv_dir = "../input/yelp-academic-photos/yelp_academic/data/business_restaurant"

In [18]:
# Define Parameters
FLAGS = {}
FLAGS['batch_size'] = 512
FLAGS['num_workers'] = 2
FLAGS['feature_extract'] = True

In [19]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [20]:
def initialize_model(model_name, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        modules = list(model_ft.children())[:-1]
        model_ft = nn.Sequential(*modules)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        new_classifier = nn.Sequential(*list(model_ft.classifier.children())[:-2])
        model_ft.classifier = new_classifier
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier = model_ft.classifier[:-3]
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Identity()
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [22]:
img_transforms = transforms.Compose([
                 transforms.Resize(224),
                 transforms.CenterCrop(224),
                 transforms.ToTensor(),
                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [23]:
class IsRestaurantDataset(Dataset):
    """Is It A Restaurant Dataset."""

    def __init__(self, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.img_names = os.listdir(root_dir)
        self.transform = transform

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.img_names[idx].split('.')[0]
        img_path = os.path.join(self.root_dir,
                                img_name + '.jpg')
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return image, img_name

In [24]:
trans_dataset = IsRestaurantDataset(photo_dir, img_transforms)
# Create training and validation dataloaders
trans_loader = torch.utils.data.DataLoader(trans_dataset, batch_size=FLAGS['batch_size'], shuffle=False, num_workers=FLAGS['num_workers'])

In [25]:
os.chdir(r'../working')

In [26]:
def extract_trf_features(model_name):
    model_ft, input_size = initialize_model(model_name, FLAGS['feature_extract'], use_pretrained=True)
    model_ft.to(device)
    
    feat_dict = {}
    with torch.no_grad():
        for num, data in enumerate(tqdm(trans_loader)):
            images, img_names = data
            images = images.cuda()

            vectors = model_ft(images)
            for idx in range(len(vectors)):
                vector = vectors[idx].cpu().numpy()
                feat_dict[img_names[idx]] = vector
    
    np.savez(f'{model_name}_features.npz', feat_dict)

In [27]:
extract_trf_features('vgg')

  0%|          | 0/391 [00:00<?, ?it/s]

In [28]:
from IPython.display import FileLink
FileLink(r'vgg_features.npz')

/kaggle/working/vgg_features.npz